In [ ]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
Data_frame = pd.read_csv(url)

In [ ]:

def Filter_Dataset(parameter):
    
    filteredDB = Data_frame.loc[Data_frame.continent == "Europe", ['continent','location','date', parameter]]
    filteredDB = filteredDB.fillna(value = 0)
    filteredDB = filteredDB.reset_index()
    return filteredDB

def Activating_the_date_column (dataframe):
    #now we can do operations using the date column in this data frame
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    return dataframe


def setting_end_date (date_start, date_end, time_frame):
    new_date_end = date_start + pd.Timedelta(time_frame)
    if new_date_end > date_end:
        new_date_end = date_end
    return new_date_end

def time_sliced_database(date_start, new_date_end, time_col_active_database):

    time_frame_Data = time_col_active_database[time_col_active_database['date'].between(date_start, new_date_end)]
    return time_frame_Data

    
def Grouping_Dataset(dataset, label):
    return dataset.groupby(label)

def Restructuring_Dataset(grouped_dataset, parameter):
     
    superfinalDB = None
    
    #counter variable is here to prevent the merging of a null database at the starting
    count = 1
    for country, country_df in grouped_dataset:
        if count == 1:
            
            #getting the maximum parameter value from each country and dividing it by that value
            superfinalDB = country_df[['date',parameter]]
            country_new_case_maximum = superfinalDB[parameter].max(axis = 0)
            superfinalDB[parameter] = superfinalDB[parameter]/country_new_case_maximum
            
            #storing the new normalised column in our final database
            superfinalDB = superfinalDB.rename(columns = {parameter: country })
            count = count + 1
        
        else:
            #getting the maximum parameter value from each country and dividing it by that value
            currDB = country_df[['date',parameter]]
            country_new_case_maximum = currDB[parameter].max(axis = 0)
            currDB[parameter] = currDB[parameter]/country_new_case_maximum
            
            #merging the new normalised column of a new country in our final database
            currDB = currDB.rename(columns = {parameter: country })
            superfinalDB = pd.merge(currDB, superfinalDB, on = 'date')
           
    return superfinalDB

def Data_Skipping(database, days_to_skip):
    
    database.fillna(value = 0)
    database.reset_index()
    count = 0
    for index, row in database.iterrows(): 
        if count%days_to_skip != 0:
            database = database.drop(index)
        count+=1
    return database

def Plot_data_frame(superfinalDB):    
    
    plt.figure(figsize=(8,5))
    plt.plot()
    for country in superfinalDB:
        if(country!= 'date'):
            plt.plot(superfinalDB.date, superfinalDB[country])
    plt.xlabel('Dates')
    plt.ylabel('Number of ' + parameter)
    plt.xticks(superfinalDB.date[::3])
    plt.title(parameter)
    plt.legend(bbox_to_anchor = (1.0,1.0))
   
        
    
    


In [ ]:
parameter = 'icu_patients'
label = 'location'

#filtering according to the required parameter
filtered_dataset = Filter_Dataset(parameter)

#enabling us to do operations using date column of the data frame
time_col_active_database = Activating_the_date_column(filtered_dataset)


#finding the value of the lowest (starting) date from the dataset
date_start = time_col_active_database['date'].min() 

#finding the value of the max (last) date from the dataset
date_end = time_col_active_database['date'].max() 

#length of each frame window
time_frame = "120 days" 

# running the loop till the last date in the dataset
while date_start < date_end:

    #finding the end date based on the user defined time frame 
    new_date_end = setting_end_date(date_start, date_end, time_frame)
    #slicing data based on time interval (start date, end date) to select a window of database interval
    time_interval_window_database = time_sliced_database( date_start, new_date_end, time_col_active_database )
  
    #grouping information from same countries
    grouped_dataset = Grouping_Dataset(time_interval_window_database, label)
    
    #restructuring the dateset with dates as rows and countries as columns
    final_form_dataset = Restructuring_Dataset(grouped_dataset, parameter)
    
    #ploting less data to analyse the graphs more clearly
    days_to_skip = 7
    truncated_database = Data_Skipping(final_form_dataset, days_to_skip)
    
    #plotting the data frame
    Plot_data_frame(truncated_database)
    
    #updating the starting point of the new window frame
    date_start = new_date_end 
    
    #In some graphs, the time interval can be less the 120 days (even after specifying it in the code)
    #This is because during merging we take the union of all the datasets from 40 European countries; it can happen that some
    #countries don't have data of some specific time intervals. Merge command just drops the values of all these dates from
    #the data frame.